The goal of this notebook is to retrieve the times of the 88D and the kas that are the same so I can determine the proper distance between the two (lat/lon).

In [1]:
# import necessary stuff
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import glob
import coords
import xarray as xr
import pyart
from datetime import datetime as dt
from datetime import timedelta
from radarcalc import *
import matplotlib.pyplot as plt
import metpy.calc as mpcalc
import metpy
import metpy.plots
from metpy.units import units
import cartopy.crs as ccrs
import cartopy
import gc

/var/folders/4f/9s_4__q16tx5t8vmww4mv03h0000gn/T/ipykernel_76402/4185104306.py:4: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd



## You are using the Python ARM Radar Toolkit (Py-ART), an open source
## library for working with weather radar data. Py-ART is partly
## supported by the U.S. Department of Energy as part of the Atmospheric
## Radiation Measurement (ARM) Climate Research Facility, an Office of
## Science user facility.
##
## If you use this software to prepare a publication, please cite:
##
##     JJ Helmus and SM Collis, JORS 2016, doi: 10.5334/jors.119



In [2]:
# gps data for first ka
ka1gps = pd.read_csv('/Users/juliabman/Desktop/research2024/GPS_Ka1_20220523.txt')
# scan data for first ka
ka1scan = pd.read_csv('/Users/juliabman/Desktop/research2024/20220523_Ka1_scan_log.txt', dtype=str)

# dealiased vad data created from VADS.ipynb for ka 1, but not bearing corrected
vad_fileska1 = sorted(glob.glob('/Users/juliabman/Desktop/research2024/product_raw_ka1/*.nc'))

# sanity checks to make sure they're loading in:
#print(vad_fileska1)
#print(ka1gps)

In [3]:
ka1gps['hhmmss[UTC]']

0        183124
1        183125
2        183126
3        183127
4        183128
          ...  
31901     32353
31902     32354
31903     32355
31904     32356
31905     32357
Name: hhmmss[UTC], Length: 31906, dtype: int64

In [4]:
file = vad_fileska1[10]

In [5]:
file

'/Users/juliabman/Desktop/research2024/product_raw_ka1/Ka1220523222620.nc'

In [6]:
radar = pyart.io.read_cfradial(file)
radar.latitude['data']

masked_array(data=[43.53127012],
             mask=False,
       fill_value=1e+20)

In [7]:
vad_ka1_times = []
for time_grab in range(len(vad_fileska1)):
    file = vad_fileska1[time_grab]
    time_yoink = file[-15:-3]
    time_yoink_datetime = dt.strptime(time_yoink, '%y%m%d%H%M%S')
    vad_ka1_times.append(time_yoink_datetime)

In [8]:
vad_ka1_times

[datetime.datetime(2022, 5, 23, 20, 56, 32),
 datetime.datetime(2022, 5, 23, 20, 57, 1),
 datetime.datetime(2022, 5, 23, 20, 57, 30),
 datetime.datetime(2022, 5, 23, 20, 57, 59),
 datetime.datetime(2022, 5, 23, 20, 58, 28),
 datetime.datetime(2022, 5, 23, 20, 58, 57),
 datetime.datetime(2022, 5, 23, 20, 59, 26),
 datetime.datetime(2022, 5, 23, 20, 59, 55),
 datetime.datetime(2022, 5, 23, 22, 17, 18),
 datetime.datetime(2022, 5, 23, 22, 25, 51),
 datetime.datetime(2022, 5, 23, 22, 26, 20),
 datetime.datetime(2022, 5, 23, 22, 26, 49),
 datetime.datetime(2022, 5, 23, 22, 27, 18),
 datetime.datetime(2022, 5, 23, 22, 27, 47),
 datetime.datetime(2022, 5, 23, 22, 28, 16),
 datetime.datetime(2022, 5, 23, 22, 28, 45),
 datetime.datetime(2022, 5, 23, 22, 29, 14),
 datetime.datetime(2022, 5, 23, 22, 29, 43),
 datetime.datetime(2022, 5, 23, 22, 30, 13),
 datetime.datetime(2022, 5, 23, 22, 30, 42),
 datetime.datetime(2022, 5, 23, 22, 31, 11),
 datetime.datetime(2022, 5, 23, 22, 31, 40),
 datetime.d

In [9]:
# get the time of the file by grabbing the corresponding numbers from the file name
file[-15:-3]
file_dt = dt.strptime(file[-15:-3], '%y%m%d%H%M%S') # converts numbers to datetime
file_dt

datetime.datetime(2022, 5, 24, 0, 3, 9)

In [10]:
###### data for ka 2

ka2gps = pd.read_csv('/Users/juliabman/Desktop/research2024/GPS_Ka2_20220523.txt')
# scan data for first ka
ka2scan = pd.read_csv('/Users/juliabman/Desktop/research2024/20220523_Ka2_scan_log.txt', dtype=str)                    

# dealiased vad data created from VADS.ipynb for ka 2, but not bearing corrected
vad_fileska2 = sorted(glob.glob('/Users/juliabman/Desktop/research2024/product_raw_ka2/*.nc'))

# sanity checks to make sure they're loading in:
#print(vad_fileska2)
#print(ka2gps)

In [34]:
ka2gps['hhmmss[UTC]']

0        200239
1        200240
2        200241
3        200242
4        200243
          ...  
22101     25355
22102     25356
22103     25357
22104     25358
22105     25359
Name: hhmmss[UTC], Length: 22106, dtype: int64

In [70]:
tobac_file = '/Users/juliabman/Desktop/research2024/tobac_Save/Track.nc'
tobac_features_xr = xr.open_dataset(tobac_file)

In [71]:
tobac_features_xr

<xarray.Dataset>
Dimensions:                  (index: 851)
Coordinates:
  * index                    (index) int64 0 1 2 3 4 5 ... 846 847 848 849 850
Data variables: (12/15)
    frame                    (index) int64 ...
    idx                      (index) int64 ...
    hdim_1                   (index) float64 ...
    hdim_2                   (index) float64 ...
    num                      (index) int64 ...
    threshold_value          (index) int64 ...
    ...                       ...
    projection_y_coordinate  (index) float64 ...
    projection_x_coordinate  (index) float64 ...
    latitude                 (index) float64 ...
    longitude                (index) float64 ...
    cell                     (index) int64 ...
    time_cell                (index) timedelta64[ns] ...

In [72]:
# want to look at how frequently tobac times are taken
tobac_time_sample = tobac_features_xr['timestr'].values[40:60]

In [73]:
tobac_time_sample

array(['2022-05-23 20:38:29', '2022-05-23 20:38:29',
       '2022-05-23 20:38:29', '2022-05-23 20:42:59',
       '2022-05-23 20:42:59', '2022-05-23 20:42:59',
       '2022-05-23 20:47:31', '2022-05-23 20:47:31',
       '2022-05-23 20:47:31', '2022-05-23 20:47:31',
       '2022-05-23 20:52:02', '2022-05-23 20:52:02',
       '2022-05-23 20:52:02', '2022-05-23 20:56:32',
       '2022-05-23 20:56:32', '2022-05-23 20:56:32',
       '2022-05-23 20:56:32', '2022-05-23 21:01:16',
       '2022-05-23 21:01:16', '2022-05-23 21:01:16'], dtype='<U19')

In [74]:
# new = first_tobac_time[-8:]

In [75]:
# split = new.split(':')

In [76]:
# ''.join(split)

In [77]:
all_tobac_times = tobac_features_xr['timestr']

In [78]:
all_tobac_times

<xarray.DataArray 'timestr' (index: 851)>
array(['2022-05-23 20:04:20', '2022-05-23 20:04:20', '2022-05-23 20:04:20',
       ..., '2022-05-24 03:59:58', '2022-05-24 03:59:58',
       '2022-05-24 03:59:58'], dtype='<U19')
Coordinates:
  * index    (index) int64 0 1 2 3 4 5 6 7 8 ... 843 844 845 846 847 848 849 850

In [79]:
# np.abs((all_tobac_times - vad_ka1_times).total_seconds()) <= 300

Create a loop that will take the HH : MM : SS of each tobac time and put them in HHMMSS format.

In [116]:
tobac_times_list = []
for time in range(len(all_tobac_times)):
    file = all_tobac_times.values[time]
    time_select = file[-8:-3] # select the elements that represent the time
    time_split = time_select.split(':') # remove the : from the selected times
    time_join = ''.join(time_split) # join the time indeces with nothing separating them (aka, with '' joining them)
    tobac_times_list.append(time_join)

In [117]:
times_list[0:5] # want only the hours and minutes, seconds is too precise 

['2004', '2004', '2004', '2004', '2004']

In [118]:
np.size(tobac_times_list)

851

In [119]:
first_vad = xr.open_dataset(vad_fileska1[0])

In [120]:
first_vad

<xarray.Dataset>
Dimensions:                    (time: 2038, range: 555, sweep: 2)
Coordinates:
  * time                       (time) datetime64[ns] 2022-05-23T20:56:34 ... ...
  * range                      (range) float32 0.0 18.0 ... 9.954e+03 9.972e+03
    azimuth                    (time) float32 ...
    elevation                  (time) float32 ...
Dimensions without coordinates: sweep
Data variables: (12/27)
    total_power                (time, range) float32 ...
    reflectivity               (time, range) float32 ...
    velocity                   (time, range) float32 ...
    spectrum_width             (time, range) float32 ...
    corrected_velocity         (time, range) float32 ...
    normalized_coherent_power  (time, range) float32 ...
    ...                         ...
    longitude                  float64 ...
    altitude                   float64 ...
    time_coverage_start        |S32 ...
    time_coverage_end          |S32 ...
    time_reference             |S32 ...
    volume_number              int32 ...
Attributes: (12/15)
    Conventions:             CF/Radial instrument_parameters
    version:                 1.3
    title:                   
    institution:             
    references:              
    source:                  
    ...                      ...
    sigmet_task_name:        AF_VAD      
    sigmet_extended_header:  false
    time_ordered:            none
    rays_missing:            8
    history:                 
    field_names:             total_power, reflectivity, velocity, spectrum_wi...

In [87]:
first_vad['time']

<xarray.DataArray 'time' (time: 2038)>
array(['2022-05-23T20:56:34.000000000', '2022-05-23T20:56:34.000000000',
       '2022-05-23T20:56:34.000000000', ..., '2022-05-23T20:56:58.000000000',
       '2022-05-23T20:56:58.000000000', '2022-05-23T20:56:58.000000000'],
      dtype='datetime64[ns]')
Coordinates:
  * time       (time) datetime64[ns] 2022-05-23T20:56:34 ... 2022-05-23T20:56:58
    azimuth    (time) float32 ...
    elevation  (time) float32 ...
Attributes:
    long_name:      time_in_seconds_since_volume_start
    standard_name:  time
    comment:        Coordinate variable for time. Time at the center of each ...

In [121]:
times_in_both_files = []
times_in_both_files_vads = []
for the_file in vad_fileska1:
    if (the_file[-9:-5] in tobac_times_list) != True:   #makes sure the times are the same from both files
        continue
        
    else:
        file_value = the_file[-9:-5]
        print(file_value)
        times_in_both_files.append(file_value)
        times_in_both_files_vads.append(the_file)

2056
2225
2230
2230
2240
2328
2328
0003


In [122]:
times_in_both_files

['/Users/juliabman/Desktop/research2024/product_raw_ka1/Ka1220523205632.nc',
 '/Users/juliabman/Desktop/research2024/product_raw_ka1/Ka1220523222551.nc',
 '/Users/juliabman/Desktop/research2024/product_raw_ka1/Ka1220523223013.nc',
 '/Users/juliabman/Desktop/research2024/product_raw_ka1/Ka1220523223042.nc',
 '/Users/juliabman/Desktop/research2024/product_raw_ka1/Ka1220523224032.nc',
 '/Users/juliabman/Desktop/research2024/product_raw_ka1/Ka1220523232822.nc',
 '/Users/juliabman/Desktop/research2024/product_raw_ka1/Ka1220523232852.nc',
 '/Users/juliabman/Desktop/research2024/product_raw_ka1/Ka1220524000309.nc']

In [123]:
np.size(times_in_both_files)

8

Trying to retrieve tobac times that match the times in both files so that i can get their lat and lon. Is it not working because the entire vad file name is in times_in_both_files?

In [124]:
times_in_both_files_tobac = []
for tobac_time in tobac_times_list:
    if (tobac_time in times_in_both_files) != True:   #makes sure the times are the same from both files
        continue
        
    else:
        file_value = tobac_time
        print(tobac_time)
        
        times_in_both_files_tobac.append(tobac_time)
#print(len(times_in_both_files))

In [126]:
times_in_both_files_tobac

[]

In [127]:
np.size(times_in_both_files_tobac)

0

Read the lats and lons from the times in both files.

In [1]:
latitude_times_in_both_files = []
longitude_times_in_both_files = []
for read_netcdf in times_in_both_files:
    read = pyart.io.read_cfradial(read_netcdf)
    latitude = read.latitude['data']
    longitude = read.longitude['data']
    latitude_times_in_both_files.append(latitude)
    longitude_times_in_both_files.append(longitude)

NameError: name 'times_in_both_files' is not defined